# Q1. Viterbi Algorithm
Write a computer program for computing the messages m1:1, m1:2, m1:3, m1:4, m1:5,
and the most likely sequence of states in Figure 15.5. See slides #16-24 in “Chapter15 Probabilistic
Reasoning over Time.pdf” for detailed calculations.

### Implementation details
I implemented a viterbi algorithm to find the messages m1:1, m1:2, m1:3, m1:4, m1:5 and likely sequence of hidden states (rain or no rain) given Observations sequence for umbrella {True, True, True, False, True}.

To avoid many lines of codes which can be hard to understand for the main implementation, I created a viterbi_utils python file that contains basic calculations such as vector addition, pairwise matrix multiplication, etc. So, to reproduce the results, Make sure to import all functions from viterbi_utils or implement them yourself. The implementation will work perfectly with Python3 but it can be modified to fit other versions. 

Below are implementation steps:


### Step1: Creating an Hidden Markov Model
First I created an HMM class which takes takes transion probability and sensor model matrices, and priors (if neccessary) as its inputs. These matrices will be input to our viterbi computation

In [1]:
import numpy as np
from viterbi_utils import *
# from probability import backward,forward
class HMM_Model:

    def __init__(self, trans_matrix, sensor_matrix, prior=None):
        self.trans_matrix= trans_matrix
        self.sensor_matrix = sensor_matrix
        
        # Initialize the prior or otherwise there will be uniformly distributed
        self.prior = prior or [0.5, 0.5]

    def sensor_dist(self, ev):
        if ev is True:
            return self.sensor_matrix[0]
        else:
            return self.sensor_matrix[1]


### Step2: Computing Forward back ward messages

In [2]:
def Forward_message(HMM, fv, ev):
    prediction = vector_add(scalar_vector_product(fv[0], HMM. trans_matrix[0]),
                            scalar_vector_product(fv[1], HMM. trans_matrix[1]))
    sensor_dist = HMM.sensor_dist(ev)

    return normalize(element_wise_product(sensor_dist, prediction))


def Backward_message(HMM, b, ev):
    sensor_dist = HMM.sensor_dist(ev)
    prediction = element_wise_product(sensor_dist, b)

    return normalize(vector_add(scalar_vector_product(prediction[0], HMM. trans_matrix[0]),
                                scalar_vector_product(prediction[1], HMM. trans_matrix[1])))


### Step3: Viterbi algorithm
Below is the actual implementation of viterbi algorithm following $Equation 15.11$ from the book

In [8]:
def viterbi_compute(HMM, ev):
    t = len(ev)
    ev = ev.copy()
    ev.insert(0, None)

    m = [[0.0, 0.0] for _ in range(len(ev) - 1)]

    # Initialiaze recursion with the first message m1
    m[0] = Forward_message(HMM, HMM.prior, ev[1])
    
    # We back track the state status  at each t-1.
    m_back_tracking= []

    for i in range(1, t):
        ## Message at time t
        m[i] = element_wise_product(HMM.sensor_dist(ev[i + 1]),
                                    [max(element_wise_product(HMM.trans_matrix[0], m[i - 1])),
                                     max(element_wise_product(HMM.trans_matrix[1], m[i - 1]))])
        ## Keep track of message at t-1
        m_back_tracking.append([np.argmax(element_wise_product(HMM.trans_matrix[0], m[i - 1])),
                                   np.argmax(element_wise_product(HMM.trans_matrix[1], m[i - 1]))])

    # Initialize Message probability 
    message_probs= [0.0] * (len(ev) - 1)
    
    # Initialize most likely path sequence
    message_path = [True] * (len(ev) - 1)
    

    # We start from the final state with largest probability and move backward while keeping track for
    # each time t the processor state xt-1 that maxmize probability of xt

    i_max = np.argmax(m[-1])

    for i in range(t - 1, -1, -1):
        message_probs[i] = m[i][i_max]
        message_path[i] = True if i_max == 0 else False
        if i > 0:
            i_max = m_back_tracking[i - 1][i_max]

    return message_path, message_probs

## Final: Results

In [9]:
state_transition_mat= [[0.7, 0.3], [0.3, 0.7]]
sensor_mat= [[0.9, 0.2], [0.1, 0.8]]
hmm = HMM_Model(state_transition_mat, sensor_mat)

In [10]:
ummbrella_evidence= [True, True, False, True, True]

# rounder(viterbi(umbrellaHMM, umbrella_evidence))
viterbi_compute(hmm, ummbrella_evidence)

([True, True, False, True, True],
 [0.8181818181818181,
  0.5154545454545454,
  0.12370909090909088,
  0.03340145454545454,
  0.02104291636363636])

In [ ]:
import numpy as np
observations=[1,1,0,1,1]
umbrella_transition =np.array( [[0.7, 0.3], [0.3, 0.7]])
umbrella_sensor = np.array([[0.9, 0.2], [0.1, 0.8]])

In [ ]:


def viterbi(y, A, B, Pi=None):
    """
    Return the MAP estimate of state trajectory of Hidden Markov Model.

    Parameters
    ----------
    y : array (T,)
        Observation state sequence. int dtype.
    A : array (K, K)
        State transition matrix. See HiddenMarkovModel.state_transition  for
        details.
    B : array (K, M)
        Emission matrix. See HiddenMarkovModel.emission for details.
    Pi: optional, (K,)
        Initial state probabilities: Pi[i] is the probability x[0] == i. If
        None, uniform initial distribution is assumed (Pi[:] == 1/K).

    Returns
    -------
    x : array (T,)
        Maximum a posteriori probability estimate of hidden state trajectory,
        conditioned on observation sequence y under the model parameters A, B,
        Pi.
    T1: array (K, T)
        the probability of the most likely path so far
    T2: array (K, T)
        the x_j-1 of the most likely path so far
    """
    # Cardinality of the state space
    K = A.shape[0]
    # Initialize the priors with default (uniform dist) if not given by caller
    Pi = Pi if Pi is not None else np.full(K, 1 / K)
    T = len(y)
    T1 = np.empty((K, T), 'd')
    T2 = np.empty((K, T), 'B')

    # Initilaize the tracking tables from first observation
    T1[:, 0] = Pi * B[:, y[0]]
    T2[:, 0] = 0

    # Iterate throught the observations updating the tracking tables
    for i in range(1, T):
        T1[:, i] = np.max(T1[:, i - 1] * A.T * B[np.newaxis, :, y[i]].T, 1)
        T2[:, i] = np.argmax(T1[:, i - 1] * A.T, 1)

    # Build the output, optimal model trajectory
    x = np.empty(T, 'B')
    x[-1] = np.argmax(T1[:, T - 1])
    for i in reversed(range(1, T)):
        x[i - 1] = T2[x[i], i]

    return x, T1, T2

In [ ]:
x,t1,t2=viterbi(observations,umbrella_transition,umbrella_sensor)
print(x)
print(t1)
print(t2)


In [ ]:
import numpy as np


def viterbi_path(prior, transmat, obslik, scaled=True, ret_loglik=False):
    '''Finds the most-probable (Viterbi) path through the HMM state trellis
    Notation:
        Z[t] := Observation at time t
        Q[t] := Hidden state at time t
    Inputs:
        prior: np.array(num_hid)
            prior[i] := Pr(Q[0] == i)
        transmat: np.ndarray((num_hid,num_hid))
            transmat[i,j] := Pr(Q[t+1] == j | Q[t] == i)
        obslik: np.ndarray((num_hid,num_obs))
            obslik[i,t] := Pr(Z[t] | Q[t] == i)
        scaled: bool
            whether or not to normalize the probability trellis along the way
            doing so prevents underflow by repeated multiplications of probabilities
        ret_loglik: bool
            whether or not to return the log-likelihood of the best path
    Outputs:
        path: np.array(num_obs)
            path[t] := Q[t]
    '''
    num_hid = obslik.shape[0] # number of hidden states
    num_obs = obslik.shape[1] # number of observations (not observation *states*)

    # trellis_prob[i,t] := Pr((best sequence of length t-1 goes to state i), Z[1:(t+1)])
    trellis_prob = np.zeros((num_hid,num_obs))
    # trellis_state[i,t] := best predecessor state given that we ended up in state i at t
    trellis_state = np.zeros((num_hid,num_obs), dtype=int) # int because its elements will be used as indicies
    path = np.zeros(num_obs, dtype=int) # int because its elements will be used as indicies

    trellis_prob[:,0] = prior * obslik[:,0] # element-wise mult
    if scaled:
        scale = np.ones(num_obs) # only instantiated if necessary to save memory
        scale[0] = 1.0 / np.sum(trellis_prob[:,0])
        trellis_prob[:,0] *= scale[0]

    trellis_state[:,0] = 0 # arbitrary value since t == 0 has no predecessor
    for t in range(1, num_obs):
        for j in range(num_hid):
            trans_probs = trellis_prob[:,t-1] * transmat[:,j] # element-wise mult
            trellis_state[j,t] = trans_probs.argmax()
            trellis_prob[j,t] = trans_probs[trellis_state[j,t]] # max of trans_probs
            trellis_prob[j,t] *= obslik[j,t]
        if scaled:
            scale[t] = 1.0 / np.sum(trellis_prob[:,t])
            trellis_prob[:,t] *= scale[t]

    path[-1] = trellis_prob[:,-1].argmax()
    for t in range(num_obs-2, -1, -1):
        path[t] = trellis_state[(path[t+1]), t+1]

    if not ret_loglik:
        return path
    else:
        if scaled:
            loglik = -np.sum(np.log(scale))
        else:
            p = trellis_prob[path[-1],-1]
            loglik = np.log(p)
        return path, loglik


if __name__=='__main__':
    # Assume there are 3 observation states, 2 hidden states, and 5 observations
    priors = np.array([0.5, 0.5])
    transmat = np.array([
        [0.7, 0.3],
        [0.7, 0.7]])
    emmat = np.array([
        [0.9, 0.2],
        [0.1, 0.8]])
    observations = np.array([1,1,1,0,1], dtype=int)
    obslik = np.array([emmat[:,z] for z in observations]).T
    print (viterbi_path(priors, transmat, obslik))                             
    print (viterbi_path(priors, transmat, obslik, scaled=False))                 
    print (viterbi_path(priors, transmat, obslik, ret_loglik=True))               
    print (viterbi_path(priors, transmat, obslik, scaled=False, ret_loglik=True)) 

In [ ]:
#=> [0 1 1 1 0]
#=> [0 1 1 1 0]
#=> (array([0, 1, 1, 1, 0]), -7.776472586614755)
#=> (array([0, 1, 1, 1, 0]), -8.0120386579275227)